### Upload Libraries Required

In [1]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from folium import plugins

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns

# import k-means from clustering stage
from sklearn.cluster import KMeans


print('Libraries imported.')

Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==2020.07=py38_0
done

## Package Plan ##

  environment location: /home/cris/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _anaconda_depends-2020.07  |           py38_0           6 KB
    anaconda-custom            |           py38_1          35 KB
    certifi-2020.12.5          |   py38h578d9bd_0         143 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         184 KB

The following NEW packages will be INSTALLED:

  _anaconda_depends  pkgs/main/linux-64::_anaconda_depends-2020.07-py38_0
  pip                conda-forge/noarch::pip-20.3.3-pyhd8ed1ab_0

The following packa

In [4]:
#Read the crs of the shp
import fiona
from os.path import expanduser

home = expanduser('~')

c = fiona.open(home + "/Data_Scientist/Whery/Pamplona/Pamplona_barrios/URBA_Pol_Barrios.shp")
crs = c.crs
crs

{'init': 'epsg:25830'}

In [55]:
#load data from shp routine
#coordinates conversion
import numpy as np
import pandas as pd
import shapefile as shp
import matplotlib.pyplot as plt
import seaborn as sns
import pyproj #import Proj, transform
from pyproj import Transformer
import geopandas
from shapely.geometry import Point, Polygon

sns.set(style='whitegrid', palette='pastel', color_codes=True)
sns.mpl.rc('figure', figsize=(10,6))

%matplotlib inline

myshp = open(home + "/Data_Scientist/Whery/Pamplona/Pamplona_barrios/URBA_Pol_Barrios.shp", "rb")
mydbf = open(home + "/Data_Scientist/Whery/Pamplona/Pamplona_barrios/URBA_Pol_Barrios.dbf", "rb")
#r
sf = shp.Reader(shp=myshp, dbf=mydbf, encoding='Latin1')
#sf = shp.Reader(home + "/Data_Scientist/Whery/Pamplona/Pamplona_barrios/URBA_Pol_Barrios.shp")

sf.shapeType
print('number of neighborhoods in Pamplona:', len(sf))
shapes = sf.shapes()

def read_shapefile(sf):
    #Read a shapefile into a Pandas dataframe with a 'coords' 
    #column holding the geometry information. This uses the pyshp
    #package
    fields = [x[0] for x in sf.fields][1:]
    records = sf.records()
    shps = [s.points for s in sf.shapes()]
    df = pd.DataFrame(columns=fields, data=records)
    df = df.assign(coords=shps)
    return df

df = read_shapefile(sf)



geometry = [Polygon(x) for x in df.coords]
df = df.drop(['coords'], axis=1)
gdf = geopandas.GeoDataFrame(df, crs="EPSG:25830", geometry=geometry)
gdf = gdf.to_crs(epsg=4326)

def coordCentroid(gdf): 
    centroids = []
    latitudes = []
    longitudes = []
    texts_nom = []
    #Defining Centroids for each neighborhood:
    for j, row in gdf.iterrows():
        txt_1 = ''
        if gdf['geometry'][j].geom_type == 'Polygon':
            Poly = Polygon(gdf['geometry'][j])
        else:
            if gdf['geometry'][j].geom_type == 'MultiPolygon':
                Poly = Polygon(gdf['geometry'][j][0])
        centroid = Poly.centroid.wkt 
        centroid = centroid.strip('POINT ')
        coord_cent = tuple(float(s) for s in centroid.strip('()').split())
        centroids.append(str(coord_cent))
        latitudes.append(coord_cent[1])
        longitudes.append(coord_cent[0])        

    gdf = gdf.assign(CENTROID=centroids)
    gdf = gdf.assign(latitude=latitudes)
    gdf = gdf.assign(longitude=longitudes)
    return(gdf)

        
#This plot Folium map
def plot_map(gdf, zoom_N):
    geo_json_barrios = folium.GeoJson(gdf)

    # Creating Folium Map
    meanLat = gdf['latitude'].mean()
    meanLong = gdf['longitude'].mean()
    
    map_barrios = folium.Map(location=[meanLat, meanLong], zoom_start=zoom_N)

    geo_json_barrios.add_to(map_barrios)
    #add markers with neighborhoods names
    for i, rec in gdf.iterrows():
        #geo_json_bcn_barris.add_child(folium.Popup(str(rec['NOMBRE'])))
        folium.Marker(location=(float(rec['latitude']), float(rec['longitude'])),
                      popup = rec['BARRIO'],
                      icon=folium.Icon(color='gray',icon='info-circle',prefix='fa')).add_to(map_barrios)
    return(map_barrios)

gdf = coordCentroid(gdf)

map_barrios = plot_map(gdf, 14)
map_barrios

number of neighborhoods in Pamplona: 13


In [61]:
gdf

,FEATURE,BARRIO,SECCIONES,ENLACE1,ENLACE2,ENLACE3,PDF1,PDF2,geometry,CENTROID,latitude,longitude
0,5041,Azpilagaña,"D 4 (1-26 la 23 no existe): 2, 11, 21, 25",http://www.pamplona.es/verPagina.asp?idPag=152...,http://www.pamplona.es/verPagina.asp?idPag=152...,http://www.pamplona.es/srv/opendata/verPagina....,http://www.pamplona.es/opendatadocs/LibroDem1D...,http://www.pamplona.es/opendatadocs/LibroDem2D...,"POLYGON ((-1.64408 42.80832, -1.64412 42.80746...","(-1.64779560093677, 42.79977393055057)",42.799774,-1.647796
1,5041,Buztintxuri-Euntzetxiki,"D 7 Arrotxapea (1-23): 9, 23",http://www.pamplona.es/verPagina.asp?idPag=152...,http://www.pamplona.es/verPagina.asp?idPag=152...,http://www.pamplona.es/srv/opendata/verPagina....,http://www.pamplona.es/opendatadocs/LibroDem1D...,http://www.pamplona.es/opendatadocs/LibroDem2D...,"POLYGON ((-1.67539 42.83419, -1.67498 42.83389...","(-1.677133813302837, 42.82782741831492)",42.827827,-1.677134
2,5041,Casco Viejo/Alde Zaharra,D 1 Casco Viejo (1-13 la 2 no existe): 1 a 13,http://www.pamplona.es/verPagina.asp?idPag=152...,http://www.pamplona.es/verPagina.asp?idPag=152...,http://www.pamplona.es/srv/opendata/verPagina....,http://www.pamplona.es/opendatadocs/LibroDem1D...,http://www.pamplona.es/opendatadocs/LibroDem2D...,"POLYGON ((-1.63885 42.81881, -1.63850 42.81855...","(-1.644230094478745, 42.81842749563572)",42.818427,-1.644230
3,5041,Chantrea/Txantrea,"D 6 Txantrea (1-16, la 8 no existe): 1 a 16",http://www.pamplona.es/verPagina.asp?idPag=152...,http://www.pamplona.es/verPagina.asp?idPag=152...,http://www.pamplona.es/srv/opendata/verPagina....,http://www.pamplona.es/opendatadocs/LibroDem1D...,http://www.pamplona.es/opendatadocs/LibroDem2D...,"POLYGON ((-1.63588 42.84177, -1.63563 42.84171...","(-1.630895852616532, 42.82846752380248)",42.828468,-1.630896
4,5041,Ensanche/Zabalgunea,D 2 Ensanche (1-17): 1 a 17 y D 5 Milagrosa (1...,http://www.pamplona.es/verPagina.asp?idPag=152...,http://www.pamplona.es/verPagina.asp?idPag=152...,http://www.pamplona.es/srv/opendata/verPagina....,http://www.pamplona.es/opendatadocs/LibroDem1D...,http://www.pamplona.es/opendatadocs/LibroDem2D...,"POLYGON ((-1.61785 42.81925, -1.61756 42.81916...","(-1.633558380056546, 42.81198285169803)",42.811983,-1.633558
5,5041,Ermitagaña-Mendebaldea,"D 3 San Juan (1-27): 14, 19 a 27",http://www.pamplona.es/verPagina.asp?idPag=152...,http://www.pamplona.es/verPagina.asp?idPag=152...,http://www.pamplona.es/srv/opendata/verPagina....,http://www.pamplona.es/opendatadocs/LibroDem1D...,http://www.pamplona.es/opendatadocs/LibroDem2D...,"POLYGON ((-1.67150 42.81817, -1.67103 42.81811...","(-1.671586016880924, 42.81084550122079)",42.810846,-1.671586
6,5041,Etxabakoitz,"D 4 Iturrama (1-26, la 23 no existe): 12, 13, 26",http://www.pamplona.es/verPagina.asp?idPag=152...,http://www.pamplona.es/verPagina.asp?idPag=152...,http://www.pamplona.es/srv/opendata/verPagina....,http://www.pamplona.es/opendatadocs/LibroDem1D...,http://www.pamplona.es/opendatadocs/LibroDem2D...,"POLYGON ((-1.66847 42.80166, -1.66835 42.80158...","(-1.675903813082267, 42.79940228680353)",42.799402,-1.675904
7,5041,Iturrama,"D 4 Iturrama (1-26, la 23 no existe): 1, 3 a 1...",http://www.pamplona.es/verPagina.asp?idPag=152...,http://www.pamplona.es/verPagina.asp?idPag=152...,http://www.pamplona.es/srv/opendata/verPagina....,http://www.pamplona.es/opendatadocs/LibroDem1D...,http://www.pamplona.es/opendatadocs/LibroDem2D...,"POLYGON ((-1.64931 42.80858, -1.64909 42.80857...","(-1.656703796944607, 42.80094981835963)",42.800950,-1.656704
8,5041,Mendillorri,D 8 Mendillorri (1-6): 1 a 6,http://www.pamplona.es/verPagina.asp?idPag=152...,http://www.pamplona.es/verPagina.asp?idPag=152...,http://www.pamplona.es/srv/opendata/verPagina....,http://www.pamplona.es/opendatadocs/LibroDem1D...,http://www.pamplona.es/opendatadocs/LibroDem2D...,"POLYGON ((-1.61324 42.81477, -1.61320 42.81473...","(-1.613920312588992, 42.81019155491443)",42.810192,-1.613920
9,5041,Milagrosa-Arrosadia,

In [57]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [60]:
nearby_venues = pd.DataFrame()
names=gdf['BARRIO']
latitudes=gdf['latitude']
longitudes=gdf['longitude']
venues_list=[]
LIMIT = 200
radius = 600

i = 0
for name, lat, lng in zip(names, latitudes, longitudes):
    i = i + 1
    print(name)

    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT)

    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']

    # return only relevant information for each nearby venue
    venues_list.append([(
        name, 
        lat, 
        lng, 
        v['venue']['name'], 
        v['venue']['location']['lat'], 
        v['venue']['location']['lng'],  
        v['venue']['categories'][0]['name']) for v in results])

nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['Neighborhood', 
              'Neighborhood Latitude', 
              'Neighborhood Longitude', 
              'Venue', 
              'Venue Latitude', 
              'Venue Longitude', 
              'Venue Category']

Azpilagaña
Buztintxuri-Euntzetxiki
Casco Viejo/Alde Zaharra
Chantrea/Txantrea
Ensanche/Zabalgunea
Ermitagaña-Mendebaldea
Etxabakoitz
Iturrama
Mendillorri
Milagrosa-Arrosadia
Rochapea/Arrotxapea
San Jorge-Sanduzelai
San Juan/Donibane


In [62]:
nearby_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Azpilagaña,42.799774,-1.647796,Restaurante Tokio,42.802422,-1.645429,Japanese Restaurant
1,Azpilagaña,42.799774,-1.647796,Arrocería Sobremesa,42.803841,-1.647750,Mediterranean Restaurant
2,Azpilagaña,42.799774,-1.647796,Farmacia Borda,42.804266,-1.644438,Pharmacy
3,Azpilagaña,42.799774,-1.647796,Caprabo,42.802799,-1.643741,Supermarket
4,Azpilagaña,42.799774,-1.647796,Eroski Center,42.801525,-1.645803,Grocery Store
5,Azpilagaña,42.799774,-1.647796,Eroski City,42.804271,-1.644341,Supermarket
6,Buztintxuri-Euntzetxiki,42.827827,-1.677134,Makro,42.832456,-1.674824,Supermarket
7,Buztintxuri-Euntzetxiki,42.827827,-1.677134,ITV Pamplona,42.832235,-1.679540,Auto Garage
8,Casco Viejo/Alde Zaharra,42.818427,-1.644230,Plaza del Castillo,42.816842,-1.642832,Plaza
9,Casco Viejo/Alde Zaharra,42.818427,-1.644230,Iruñazarra Bar Restaurante,42.818472,-1.643280,Restaurant


In [168]:
#Create a dictionary with number of Venues per neighborhood in Pamplona (limit = 200 venues)
temp1 = nearby_venues.groupby('Neighborhood').count()
dict_tot_F = dict(temp1['Venue'])
dict_tot_F

{'Azpilagaña': 6,
 'Buztintxuri-Euntzetxiki': 2,
 'Casco Viejo/Alde Zaharra': 60,
 'Chantrea/Txantrea': 7,
 'Ensanche/Zabalgunea': 40,
 'Ermitagaña-Mendebaldea': 11,
 'Etxabakoitz': 6,
 'Iturrama': 18,
 'Mendillorri': 7,
 'Milagrosa-Arrosadia': 10,
 'Rochapea/Arrotxapea': 15,
 'San Jorge-Sanduzelai': 3,
 'San Juan/Donibane': 11}

In [64]:
print('There are {} uniques categories.'.format(len(nearby_venues['Venue Category'].unique())))

There are 80 uniques categories.


In [65]:
nearby_venues['Venue Category'].unique()

array(['Japanese Restaurant', 'Mediterranean Restaurant', 'Pharmacy',
       'Supermarket', 'Grocery Store', 'Auto Garage', 'Plaza',
       'Restaurant', 'Pie Shop', 'Gift Shop', 'Spanish Restaurant',
       'Road', 'Church', 'Hotel', 'Tapas Restaurant', 'Historic Site',
       'Cupcake Shop', 'Café', 'Park', 'Pedestrian Plaza',
       'Sandwich Place', 'Farmers Market', 'Theater', 'Coffee Shop',
       'Bar', 'Creperie', 'Ice Cream Shop', 'BBQ Joint', 'Brewery',
       'Auditorium', 'Event Space', 'Gastropub', 'Record Shop',
       'Piercing Parlor', 'Diner', 'Gym / Fitness Center',
       'Food & Drink Shop', 'Breakfast Spot', 'Market',
       'Moroccan Restaurant', 'French Restaurant', 'Steakhouse',
       'Paella Restaurant', 'Pool Hall', 'Sports Bar', 'Burger Joint',
       'Italian Restaurant', 'Clothing Store', 'Gym Pool', 'Pizza Place',
       'Bookstore', 'Pool', 'Athletics & Sports', 'Mobile Phone Shop',
       'Asian Restaurant', 'Bakery', 'Food', 'Beer Garden', 'University'

In [69]:
nearby_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Azpilagaña,42.799774,-1.647796,Restaurante Tokio,42.802422,-1.645429,Japanese Restaurant
1,Azpilagaña,42.799774,-1.647796,Arrocería Sobremesa,42.803841,-1.647750,Mediterranean Restaurant
2,Azpilagaña,42.799774,-1.647796,Farmacia Borda,42.804266,-1.644438,Pharmacy
3,Azpilagaña,42.799774,-1.647796,Caprabo,42.802799,-1.643741,Supermarket
4,Azpilagaña,42.799774,-1.647796,Eroski Center,42.801525,-1.645803,Grocery Store
5,Azpilagaña,42.799774,-1.647796,Eroski City,42.804271,-1.644341,Supermarket
6,Buztintxuri-Euntzetxiki,42.827827,-1.677134,Makro,42.832456,-1.674824,Supermarket
7,Buztintxuri-Euntzetxiki,42.827827,-1.677134,ITV Pamplona,42.832235,-1.679540,Auto Garage
8,Casco Viejo/Alde Zaharra,42.818427,-1.644230,Plaza del Castillo,42.816842,-1.642832,Plaza
9,Casco Viejo/Alde Zaharra,42.818427,-1.644230,Iruñazarra Bar Restaurante,42.818472,-1.643280,Restaurant


In [66]:
#save result into csv:
nearby_venues.to_csv(home + '/Data_Scientist/Whery/Pamplona/nearby_venues_byNeighborhood_Foursquare.csv')

In [119]:
nearby_venues_googleT = nearby_venues_googleT.reset_index(drop=True)

In [120]:
nearby_venues_googleT

,index,Neighborhood,Neighborhood_Latitude,Neighborhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category1,Venue_Category2,Venue_Category3,Venue_Category4,Rating
0,0,Azpilagaña,42.79977393055057,-1.64779560093677,Pamplona,42.812526,-1.645774,locality,political,-,-,0.0
1,1,Azpilagaña,42.79977393055057,-1.64779560093677,Arroceria Sobremesa,42.803739,-1.647395,restaurant,food,point_of_interest,establishment,4.2
2,2,Azpilagaña,42.79977393055057,-1.64779560093677,Clínica Dental Fernando Villanueva Mesa,42.802971,-1.646432,dentist,health,point_of_interest,establishment,3.0
3,3,Azpilagaña,42.79977393055057,-1.64779560093677,Asador Almadia,42.803739,-1.647397,restaurant,food,point_of_interest,establishment,3.0
4,4,Azpilagaña,42.79977393055057,-1.64779560093677,University of Navarre: ISSA,42.800741,-1.654656,university,point_of_interest,establishment,-,4.5
5,5,Azpilagaña,42.79977393055057,-1.64779560093677,Web Design Pamplona,42.801938,-1.646147,point_of_interest,establishment,-,-,4.9
6,6,Azpilagaña,42.79977393055057,-1.64779560093677,Public school Azpilagaña,42.802184,-1.647440,school,point_of_interest,establishment,-,4.5
7,7,Azpilagaña,42.79977393055057,-1.64779560093677,ANASAPS Pamplona (Asociación Navarra para la S...,42.802307,-1.646765,health,point_of_interest,establishment,-,5.0
8,8,Azpilagaña,42.79977393055057,-1.64779560093677,"IKEI research & consultancy, S.A. Pamplona",42.801192,-1.647160,point_of_interest,establishment,-,-,0.0
9,9,Azpilagaña,42.79977393055057,-1.64779560093677,Parroquia San Raimundo de Fitero,42.801905,-1.645600,church,place_of_worship,point_of_interest,establishment,3.8


In [121]:
#save result into csv:
nearby_venues_googleT.to_csv(home + '/Data_Scientist/Whery/Pamplona/nearby_venues_byNeighborhood_Google.csv')

In [122]:
#Create a dictionary with number of Venues per neighborhood in Pamplona
temp1 = nearby_venues_googleT.groupby('Neighborhood').count()
dict_tot = dict(temp1['Venue'])
dict_tot

{'Azpilagaña': 60,
 'Buztintxuri-Euntzetxiki': 33,
 'Casco Viejo/Alde Zaharra': 60,
 'Chantrea/Txantrea': 60,
 'Ensanche/Zabalgunea': 60,
 'Ermitagaña-Mendebaldea': 60,
 'Etxabakoitz': 60,
 'Iturrama': 60,
 'Mendillorri': 60,
 'Milagrosa-Arrosadia': 60,
 'Rochapea/Arrotxapea': 60,
 'San Jorge-Sanduzelai': 58,
 'San Juan/Donibane': 60}

In [124]:
print('There are {} uniques categories.'.format(len(nearby_venues_googleT['Venue_Category1'].unique())))

There are 69 uniques categories.


In [125]:
nearby_venues_googleT['Venue_Category1'].unique()

array(['locality', 'restaurant', 'dentist', 'university',
       'point_of_interest', 'school', 'health', 'church', 'doctor',
       'finance', 'neighborhood', 'night_club', 'home_goods_store', 'bar',
       'general_contractor', 'accounting', 'food', 'bank',
       'real_estate_agency', 'beauty_salon', 'physiotherapist',
       'car_repair', 'grocery_or_supermarket', 'convenience_store',
       'lodging', 'parking', 'place_of_worship', 'jewelry_store',
       'department_store', 'book_store', 'post_office',
       'electronics_store', 'furniture_store', 'library',
       'insurance_agency', 'hospital', 'clothing_store', 'hair_care',
       'plumber', 'atm', 'funeral_home', 'gym', 'supermarket',
       'electrician', 'lawyer', 'local_government_office', 'bakery',
       'pharmacy', 'shoe_store', 'travel_agency', 'meal_delivery', 'park',
       'florist', 'laundry', 'drugstore', 'spa', 'locksmith', 'store',
       'bicycle_store', 'secondary_school', 'veterinary_care', 'stadium',
      

In [126]:
nearby_venues_googleT['Venue_Category2'].unique()

array(['political', 'food', 'health', 'point_of_interest',
       'establishment', 'place_of_worship', 'bar', 'restaurant', 'store',
       'finance', 'lodging', 'atm', 'meal_delivery', 'electronics_store',
       'cafe', 'night_club', 'clothing_store', 'home_goods_store', 'spa',
       'bank', 'grocery_or_supermarket', 'accounting',
       'general_contractor', 'doctor', 'hair_care', 'convenience_store',
       'gym', 'local_government_office', 'school', 'shoe_store',
       'transit_station'], dtype=object)

In [127]:
nearby_venues_googleT['Venue_Category3'].unique()

array(['-', 'point_of_interest', 'establishment', 'store', 'food',
       'accounting', 'parking', 'home_goods_store', 'bar',
       'clothing_store', 'general_contractor', 'finance', 'health', 'spa'],
      dtype=object)

In [128]:
nearby_venues_googleT['Venue_Category4'].unique()

array(['-', 'establishment', 'point_of_interest', 'store', 'finance',
       'cafe', 'food', 'health'], dtype=object)

In [129]:
# one hot encoding
pamplona_onehot = pd.get_dummies(nearby_venues_googleT[['Venue_Category1']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
pamplona_onehot['Neighborhood'] = nearby_venues_googleT['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [pamplona_onehot.columns[-1]] + list(pamplona_onehot.columns[:-1])
pamplona_onehot = pamplona_onehot[fixed_columns]

pamplona_onehot.head()

,Neighborhood,accounting,atm,bakery,bank,bar,beauty_salon,bicycle_store,book_store,bus_station,cafe,car_repair,cemetery,church,clothing_store,convenience_store,courthouse,dentist,department_store,doctor,drugstore,electrician,electronics_store,finance,florist,food,funeral_home,furniture_store,general_contractor,grocery_or_supermarket,gym,hair_care,hardware_store,health,home_goods_store,hospital,insurance_agency,jewelry_store,laundry,lawyer,library,local_government_office,locality,locksmith,lodging,meal_delivery,moving_company,neighborhood,night_club,park,parking,pharmacy,physiotherapist,place_of_worship,plumber,point_of_interest,post_office,real_estate_agency,restaurant,school,secondary_school,shoe_store,spa,stadium,store,supermarket,taxi_stand,travel_agency,university,veterinary_care
0,Azpilagaña,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Azpilagaña,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,Azpilagaña,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Azpilagaña,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,Azpilagaña,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [130]:
pamplona_onehot.shape

(751, 70)

In [145]:
pamplona_grouped = pamplona_onehot.groupby('Neighborhood').mean().reset_index()
pamplona_grouped['Total_venues'] = pamplona_grouped['Neighborhood'].map(dict_tot)
pamplona_grouped

,Neighborhood,accounting,atm,bakery,bank,bar,beauty_salon,bicycle_store,book_store,bus_station,cafe,car_repair,cemetery,church,clothing_store,convenience_store,courthouse,dentist,department_store,doctor,drugstore,electrician,electronics_store,finance,florist,food,funeral_home,furniture_store,general_contractor,grocery_or_supermarket,gym,hair_care,hardware_store,health,home_goods_store,hospital,insurance_agency,jewelry_store,laundry,lawyer,library,local_government_office,locality,locksmith,lodging,meal_delivery,moving_company,neighborhood,night_club,park,parking,pharmacy,physiotherapist,place_of_worship,plumber,point_of_interest,post_office,real_estate_agency,restaurant,school,secondary_school,shoe_store,spa,stadium,store,supermarket,taxi_stand,travel_agency,university,veterinary_care,Total_venues
0,Azpilagaña,0.016667,0.000000,0.000000,0.033333,0.016667,0.016667,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.016667,0.000000,0.016667,0.000000,0.033333,0.000000,0.016667,0.000000,0.000000,0.000000,0.066667,0.000000,0.016667,0.000000,0.000000,0.033333,0.016667,0.000000,0.000000,0.000000,0.066667,0.016667,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.016667,0.00,0.000000,0.000000,0.000000,0.016667,0.016667,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,0.416667,0.000000,0.016667,0.033333,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,60
1,Buztintxuri-Euntzetxiki,0.030303,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.363636,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.030303,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.484848,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,33
2,Casco Viejo/Alde Zaharra,0.000000,0.000000,0.000000,0.050000,0.050000,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667,0.016667,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,0.016667,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.033333,0.016667,0.016667,0.05,0.000000,0.000000,0.016667,0.000000,0.016667,0.00,0.216667,0.000000,0.000000,0.016667,0.000000,0.000000,0.016667,0.000000,0.000000,0.016667,0.000000,0.216667,0.016667,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,60
3,Chantrea/Txantrea,0.000000,0.033333,0.016667,0.016667,0.050000,0.050000,0.000000,0.016667,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667,0.000000,0.016667,0.000000,0.016667,0.000000,0.000000,0.000000,0.016667,0.016667,0.000000,0.066667,0.000000,0.050000,0.066667,0.000000,0.033333,0.000000,0.000000,0.000000,0.00,0.000000,0.016667,0.000000,0.016667,0.033333,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,0.033333,0.233333,0.000000,0.050000,0.016667,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,60
4,Ensanche/Zabalgunea,0.000000,0.000000,0.016667,0.066667,0.016667,0.016667,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.016667,0.000000,0.000000,0.016667,0.000000,0.016667,0.000000,0.000000,0.033333,0.016667,0.016667,0.033333,0.000000,0.016667,0.016667,0.000000,0.016667,0.000000,0.000000,0.033333,0.000000,0.000000,0.016667,0.00,0.016667,0.000000,0.000000,0.000000,0.016667,0.00,0.033333,0.016667,0.000000,0.016667,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.050000,0.083333,0.050000,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,60
5,Er

In [146]:
num_top_venues = 10

for hood in pamplona_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = pamplona_grouped[pamplona_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Azpilagaña----
                venue   freq
0        Total_venues  60.00
1   point_of_interest   0.42
2             finance   0.07
3              health   0.07
4          university   0.03
5                bank   0.03
6  general_contractor   0.03
7          restaurant   0.03
8             dentist   0.03
9          car_repair   0.03


----Buztintxuri-Euntzetxiki----
                    venue   freq
0            Total_venues  33.00
1       point_of_interest   0.48
2                    food   0.36
3                  school   0.03
4                locality   0.03
5  grocery_or_supermarket   0.03
6              accounting   0.03
7                     bar   0.03
8              book_store   0.00
9                    bank   0.00


----Casco Viejo/Alde Zaharra----
               venue   freq
0       Total_venues  60.00
1  point_of_interest   0.22
2            lodging   0.22
3         restaurant   0.10
4               bank   0.05
5                bar   0.05
6      jewelry_store   0.05
7     

In [143]:
#Function to sort venues in descending order:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [163]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venuesneighborhoods
columns = ['Neighborhood', 'Total_venues']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = pamplona_grouped['Neighborhood']
neighborhoods_venues_sorted['Total_venues'] = pamplona_grouped['Total_venues']

for ind in np.arange(pamplona_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 2:] = return_most_common_venues(pamplona_grouped.iloc[ind,0:pamplona_grouped.shape[1]-1], num_top_venues)

neighborhoods_venues_sorted.head(15)

,Neighborhood,Total_venues,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Azpilagaña,60,point_of_interest,health,finance,general_contractor,restaurant,car_repair,bank,university,dentist,night_club
1,Buztintxuri-Euntzetxiki,33,point_of_interest,food,bar,locality,school,accounting,grocery_or_supermarket,general_contractor,furniture_store,funeral_home
2,Casco Viejo/Alde Zaharra,60,point_of_interest,lodging,restaurant,bank,bar,jewelry_store,health,home_goods_store,hospital,locality
3,Chantrea/Txantrea,60,point_of_interest,general_contractor,hair_care,supermarket,bar,beauty_salon,gym,real_estate_agency,plumber,school
4,Ensanche/Zabalgunea,60,point_of_interest,restaurant,bank,real_estate_agency,school,health,car_repair,electronics_store,lodging,food
5,Ermitagaña-Mendebaldea,60,point_of_interest,general_contractor,insurance_agency,doctor,health,dentist,school,electronics_store,finance,bank
6,Etxabakoitz,60,point_of_interest,bakery,locksmith,hair_care,school,locality,home_goods_store,plumber,health,general_contractor
7,Iturrama,60,point_of_interest,university,health,real_estate_agency,beauty_salon,home_goods_store,locality,dentist,neighborhood,general_contractor
8,Mendillorri,60,point_of_interest,school,health,bakery,bank,veterinary_care,atm,supermarket,electronics_store,secondary_school
9,Milagrosa-Arrosadia,60,point_of_interest,university,health,bank,bar,stadium,electronics_store,furniture_store,school,veterinary_care


In [ ]:
#Define what the user is looking for:
supermarket
bakery
gym
bank
running

In [165]:
#Same with foursquare data:
# one hot encoding
pamplona_onehot_F = pd.get_dummies(nearby_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
pamplona_onehot_F['Neighborhood'] = nearby_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [pamplona_onehot_F.columns[-1]] + list(pamplona_onehot_F.columns[:-1])
pamplona_onehot_F = pamplona_onehot_F[fixed_columns]

pamplona_onehot_F.head()

,Neighborhood,Art Museum,Asian Restaurant,Athletics & Sports,Auditorium,Auto Garage,BBQ Joint,Bakery,Bar,Beer Garden,Big Box Store,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Café,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,Creperie,Cupcake Shop,Diner,Event Space,Farmers Market,Food,Food & Drink Shop,French Restaurant,Furniture / Home Store,Gastropub,German Restaurant,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Historic Site,Hotel,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Karaoke Bar,Market,Mediterranean Restaurant,Mexican Restaurant,Mobile Phone Shop,Moroccan Restaurant,Nightclub,Paella Restaurant,Park,Pedestrian Plaza,Pharmacy,Pie Shop,Piercing Parlor,Pizza Place,Playground,Plaza,Pool,Pool Hall,Record Shop,Restaurant,Road,Rock Club,Sandwich Place,Seafood Restaurant,Snack Place,Soccer Stadium,Spanish Restaurant,Sports Bar,Stadium,Steakhouse,Supermarket,Tapas Restaurant,Theater,Trail,University,Wings Joint
0,Azpilagaña,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Azpilagaña,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Azpilagaña,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Azpilagaña,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,Azpilagaña,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [169]:
pamplona_grouped_F = pamplona_onehot_F.groupby('Neighborhood').mean().reset_index()
pamplona_grouped_F['Total_venues'] = pamplona_grouped_F['Neighborhood'].map(dict_tot_F)
pamplona_grouped_F

,Neighborhood,Art Museum,Asian Restaurant,Athletics & Sports,Auditorium,Auto Garage,BBQ Joint,Bakery,Bar,Beer Garden,Big Box Store,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Café,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,Creperie,Cupcake Shop,Diner,Event Space,Farmers Market,Food,Food & Drink Shop,French Restaurant,Furniture / Home Store,Gastropub,German Restaurant,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Historic Site,Hotel,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Karaoke Bar,Market,Mediterranean Restaurant,Mexican Restaurant,Mobile Phone Shop,Moroccan Restaurant,Nightclub,Paella Restaurant,Park,Pedestrian Plaza,Pharmacy,Pie Shop,Piercing Parlor,Pizza Place,Playground,Plaza,Pool,Pool Hall,Record Shop,Restaurant,Road,Rock Club,Sandwich Place,Seafood Restaurant,Snack Place,Soccer Stadium,Spanish Restaurant,Sports Bar,Stadium,Steakhouse,Supermarket,Tapas Restaurant,Theater,Trail,University,Wings Joint,Total_venues
0,Azpilagaña,0.000000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.166667,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000,0.166667,0.000000,0.000000,0.000,0.000000,0.000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.0,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.0,6
1,Buztintxuri-Euntzetxiki,0.000000,0.000000,0.000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.0,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.0,2
2,Casco Viejo/Alde Zaharra,0.000000,0.000000,0.000,0.016667,0.0,0.016667,0.000000,0.050000,0.000000,0.000000,0.000000,0.0,0.000000,0.016667,0.000,0.033333,0.000000,0.016667,0.000000,0.000000,0.066667,0.000000,0.000000,0.016667,0.016667,0.016667,0.016667,0.016667,0.000000,0.000000,0.000,0.000000,0.016667,0.000000,0.033333,0.000000,0.0,0.000000,0.000000,0.016667,0.083333,0.016667,0.000000,0.000000,0.000000,0.000,0.033333,0.000000,0.000000,0.000,0.000000,0.000,0.033333,0.016667,0.000000,0.016667,0.016667,0.000,0.000000,0.083333,0.000000,0.000,0.016667,0.033333,0.033333,0.000000,0.016667,0.000000,0.000000,0.0,0.116667,0.000,0.0,0.000000,0.000000,0.100000,0.016667,0.000000,0.000000,0.0,60
3,Chantrea/Txantrea,0.000000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.0,0.142857,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000,0.000000,0.000000,0.000000,0.000000,0.142857,0.0,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.0,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.0,7
4,Ensanche/Zabalgunea,0.000000,0.000000,0.025,0.000000,0.0,0.000000,0.000000,0.050000,0.000000,0.000000,0.025000,0.0,0.000000,0.000000,0.025,0.050000,0.000000,0.000000,0.025000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00

In [170]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venuesneighborhoods
columns = ['Neighborhood', 'Total_venues']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_F = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_F['Neighborhood'] = pamplona_grouped_F['Neighborhood']
neighborhoods_venues_sorted_F['Total_venues'] = pamplona_grouped_F['Total_venues']

for ind in np.arange(pamplona_grouped_F.shape[0]):
    neighborhoods_venues_sorted_F.iloc[ind, 2:] = return_most_common_venues(pamplona_grouped_F.iloc[ind,0:pamplona_grouped_F.shape[1]-1], num_top_venues)

neighborhoods_venues_sorted_F.head(15)

,Neighborhood,Total_venues,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Azpilagaña,6,Supermarket,Mediterranean Restaurant,Grocery Store,Pharmacy,Japanese Restaurant,Food,College Gym,Creperie,Cupcake Shop,Diner
1,Buztintxuri-Euntzetxiki,2,Supermarket,Auto Garage,Wings Joint,Food,College Cafeteria,College Gym,Creperie,Cupcake Shop,Diner,Event Space
2,Casco Viejo/Alde Zaharra,60,Spanish Restaurant,Tapas Restaurant,Plaza,Hotel,Coffee Shop,Bar,Gift Shop,Park,Café,Restaurant
3,Chantrea/Txantrea,7,Gym / Fitness Center,Bar,Restaurant,Breakfast Spot,Food & Drink Shop,Grocery Store,Tapas Restaurant,Gift Shop,College Gym,Creperie
4,Ensanche/Zabalgunea,40,Spanish Restaurant,Restaurant,Plaza,Bar,Café,Pool Hall,Pool,Hotel,Pizza Place,Burger Joint
5,Ermitagaña-Mendebaldea,11,Hotel,Mobile Phone Shop,Asian Restaurant,Grocery Store,Supermarket,Bakery,Café,Farmers Market,Restaurant,Breakfast Spot
6,Etxabakoitz,6,Food,Hotel,Pool,Park,Restaurant,Beer Garden,Farmers Market,College Gym,Creperie,Cupcake Shop
7,Iturrama,18,Brewery,Bookstore,Art Museum,Snack Place,College Cafeteria,Cocktail Bar,Diner,Chinese Restaurant,Seafood Restaurant,Big Box Store
8,Mendillorri,7,Bakery,Gym / Fitness Center,BBQ Joint,Sandwich Place,Playground,Grocery Store,German Restaurant,Farmers Market,College Cafeteria,College Gym
9,Milagrosa-Arrosadia,10,Spanish Restaurant,Wings Joint,Sports Bar,College Cafeteria,Breakfast Spot,Bowling Alley,Soccer Stadium,Gym,Stadium,Food


In [173]:
#Running routes examples: http://www.runmap.net/en/
def process_gpx_to_df(file_name):
    import gpxpy
    gpx = gpxpy.parse(open(file_name)) 

    #(1)make DataFrame
    track = gpx.tracks[0]
    segment = track.segments[0]
    # Load the data into a Pandas dataframe (by way of a list)
    data = []
    segment_length = segment.length_3d()
    for point_idx, point in enumerate(segment.points): 
        data.append([point.longitude, point.latitude,point.elevation,
                     point.time, segment.get_speed(point_idx)])
    columns = ['Longitude', 'Latitude', 'Altitude', 'Time', 'Speed']
    gpx_df = pd.DataFrame(data, columns=columns)

    #2(make points tuple for line)
    points = []
    for track in gpx.tracks:
        for segment in track.segments: 
            for point in segment.points:
                points.append(tuple([point.latitude, point.longitude]))

    return gpx_df, points


In [184]:
#Create routes on map
map_run = map_barrios
filenames = [home + '/Data_Scientist/Whery/Pamplona/running/81908.gpx',
            home + '/Data_Scientist/Whery/Pamplona/running/385329.gpx',
            home + '/Data_Scientist/Whery/Pamplona/running/1367874.gpx',
            home + '/Data_Scientist/Whery/Pamplona/running/1429972.gpx',
            home + '/Data_Scientist/Whery/Pamplona/running/8Km-rio.gpx']
colors = ['red', 'orange', 'green', 'purple', 'pink']#'red', 
num = 0
for filename in filenames:
    df, points = process_gpx_to_df(filename)
    folium.PolyLine(points, color=colors[num], weight=4.5, opacity=.5).add_to(map_run)
    num = num + 1

In [210]:
# Measurement ruler icon tool to measure distances in map
from folium.plugins import FloatImage
url = ('https://media.licdn.com/mpr/mpr/shrinknp_100_100/AAEAAQAAAAAAAAlgAAAAJGE3OTA4YTdlLTkzZjUtNDFjYy1iZThlLWQ5OTNkYzlhNzM4OQ.jpg')
FloatImage(url, bottom=5, left=85).add_to(map_run)
map_run

In [211]:
#ruido
#load data from shp routine
#coordinates conversion
import numpy as np
import pandas as pd
import shapefile as shp
import matplotlib.pyplot as plt
import seaborn as sns
import pyproj #import Proj, transform
from pyproj import Transformer
import geopandas
from shapely.geometry import Point, Polygon

myshpR = open(home + "/Data_Scientist/Whery/Pamplona/RUIDOS_Pol_ZonaServiAc/RUIDOS_Pol_ZonaServiAc.shp", "rb")
mydbfR = open(home + "/Data_Scientist/Whery/Pamplona/RUIDOS_Pol_ZonaServiAc/RUIDOS_Pol_ZonaServiAc.dbf", "rb")
#r
sfR = shp.Reader(shp=myshpR, dbf=mydbfR, encoding='Latin1')
#sf = shp.Reader(home + "/Data_Scientist/Whery/Pamplona/Pamplona_barrios/URBA_Pol_Barrios.shp")

sfR.shapeType
print('number of Zonas servicio in Pamplona:', len(sfR))
shapes = sfR.shapes()

def read_shapefile(sf):
    #Read a shapefile into a Pandas dataframe with a 'coords' 
    #column holding the geometry information. This uses the pyshp
    #package
    fields = [x[0] for x in sf.fields][1:]
    records = sf.records()
    shps = [s.points for s in sf.shapes()]
    df = pd.DataFrame(columns=fields, data=records)
    df = df.assign(coords=shps)
    return df


dfR = read_shapefile(sfR)



geometryR = [Polygon(x) for x in dfR.coords]
dfR = dfR.drop(['coords'], axis=1)
gdfR = geopandas.GeoDataFrame(dfR, crs="EPSG:25830", geometry=geometryR)
gdfR = gdfR.to_crs(epsg=4326)

       
#This plot Folium map
def plot_map(gdf, meanLat, meanLong, zoom_N):
    geo_json_ruido = folium.GeoJson(gdfR)
    
    # Creating Folium Map
       
    map_ruido = folium.Map(location=[meanLat, meanLong], zoom_start=zoom_N)

    geo_json_ruido.add_to(map_ruido)

    return(map_ruido)


meanLat = gdf['latitude'].mean()
meanLong = gdf['longitude'].mean()
map_ruido = plot_map(gdfR, meanLat, meanLong, 14)
#map_ruido

number of Zonas servicio in Pamplona: 1


### Zona Buztinxuri:
<p>El punto central del barrio caía en el centro penitenciario con muy pocas venues en un radio de 600m.
A continuación tomamos otro punto (ver mapa final) y analizamos los puntos de interés en las inmediaciones (radio 1Km).</p>
<i> NOTA: con la API gratuita los puntos de interés obtenidos están limitados a 60.</i>

In [220]:
#listado de todos los puntos de interés devueltos por la API:
nearby_venues_buzt

,index,Neighborhood,Neighborhood_Latitude,Neighborhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category1,Venue_Category2,Venue_Category3,Venue_Category4,Rating
0,0,Buztintxuri_2,42.82966443913618,-1.6631849767283415,Pamplona,42.812526,-1.645774,locality,political,-,-,0.0
1,1,Buztintxuri_2,42.82966443913618,-1.6631849767283415,Hotel Pamplona Plaza,42.826076,-1.655268,lodging,point_of_interest,establishment,-,4.0
2,2,Buztintxuri_2,42.82966443913618,-1.6631849767283415,Oficina Banco BBVA,42.833258,-1.669067,bank,atm,accounting,finance,2.5
3,3,Buztintxuri_2,42.82966443913618,-1.6631849767283415,Mercadona,42.832115,-1.665992,supermarket,grocery_or_supermarket,food,point_of_interest,4.0
4,4,Buztintxuri_2,42.82966443913618,-1.6631849767283415,Almacenes San Fernando,42.826864,-1.658941,point_of_interest,establishment,-,-,0.0
5,5,Buztintxuri_2,42.82966443913618,-1.6631849767283415,Caja Rural De Navarra,42.827095,-1.664920,bank,finance,point_of_interest,establishment,2.9
6,6,Buztintxuri_2,42.82966443913618,-1.6631849767283415,Autointegral Aldabe S.L.,42.827226,-1.652753,car_repair,point_of_interest,establishment,-,4.5
7,7,Buztintxuri_2,42.82966443913618,-1.6631849767283415,Centro de masajes Pachi Marchal,42.826685,-1.662128,health,point_of_interest,establishment,-,5.0
8,8,Buztintxuri_2,42.82966443913618,-1.6631849767283415,Bar-Restaurante Manolo,42.826988,-1.659190,bar,restaurant,food,point_of_interest,4.0
9,9,Buztintxuri_2,42.82966443913618,-1.6631849767283415,Estudio Fotográfico Pujol Pamplona | Fotografí...,42.830990,-1.663536,point_of_interest,establishment,-,-,5.0


In [262]:
#Representación gráfica de los puntos de interés obtenidos:
import folium
import numpy as np
#from geopy.distance import geodesic
from re import search
import fontawesome as fa

map_buzt = folium.Map(location=[42.82966443913618,-1.6631849767283415], zoom_start=16)


folium.Circle(location = (42.82966443913618,-1.6631849767283415),
                    radius = 1000,
                    color = '#3186cc',
                    fill = True,
                    fill_color = '#3186cc').add_to(map_buzt)
#add markers with venues names
#<i class="fas fa-shopping-cart"></i>
#<i class="fas fa-piggy-bank"></i>
#<i class="fas fa-glass-cheers"></i>
#<i class="fas fa-bed"></i>
#<i class="fas fa-prescription-bottle-alt"></i>
#<i class="fas fa-gavel"></i>
#<i class="fas fa-bread-slice"></i>
#<i class="fas fa-school"></i>
#<i class="fas fa-hand-holding-medical"></i>
#<i class="fas fa-tooth"></i>
for i, rec in nearby_venues_buzt.iterrows():
    isIcon = False
    if isIcon:
        iconi = 'info-circle'
        prefixi = 'fa'
    folium.Marker(location=(float(rec['Venue_Latitude']), float(rec['Venue_Longitude'])),
                  popup = rec['Venue'] + ': [' + rec['Venue_Category1'] + ',' + rec['Venue_Category2'] + 
                  ',' + rec['Venue_Category3'] + ',' + rec['Venue_Category4'] + ']',
                  icon=folium.Icon(icon=iconi,prefix=prefixi)).add_to(map_buzt) #color='gray',
'''
    if search('supermarket', rec['Venue_Category1']):
    #if rec['Venue_Category1'] == 'supermarket':
        iconi = 'shopping-cart'
        prefixi = 'fa'
        isIcon = True
    #if rec['Venue_Category1'] == 'bank':
    if search('bank', rec['Venue_Category1']):
        iconi = fa.icons['piggy-bank']
        prefixi = 'fa'
        isIcon = True
    if rec['Venue_Category1'] == 'bar':
        iconi = 'glass-cheers'
        prefixi = 'fa'
        isIcon = True
    if rec['Venue_Category1'] == 'lodging':
        iconi = 'bed'
        prefixi = 'fa'
        isIcon = True
    if rec['Venue_Category1'] == 'pharmacy':
        iconi = 'prescription-bottle-alt'
        prefixi = 'fa'
        isIcon = True
    if rec['Venue_Category1'] == 'lawyer':
        iconi = 'gavel'
        prefixi = 'fa'
        isIcon = True
    if rec['Venue_Category1'] == 'bakery':
        iconi = 'bread-slice'
        prefixi = 'fa'
        isIcon = True
    if rec['Venue_Category1'] == 'school':
        iconi = 'book-open'
        prefixi = 'fa'
        #iconi = 'basket'
        #prefixi = 'glyphicon'
        isIcon = True
    if rec['Venue_Category1'] == 'health':
        iconi = 'hand-holding-medical'
        prefixi = 'fa'
        isIcon = True
    if rec['Venue_Category1'] == 'dentist':
        iconi = 'tooth'
        prefixi = 'fa'
        isIcon = True
'''

"\n    if search('supermarket', rec['Venue_Category1']):\n    #if rec['Venue_Category1'] == 'supermarket':\n        iconi = 'shopping-cart'\n        prefixi = 'fa'\n        isIcon = True\n    #if rec['Venue_Category1'] == 'bank':\n    if search('bank', rec['Venue_Category1']):\n        iconi = fa.icons['piggy-bank']\n        prefixi = 'fa'\n        isIcon = True\n    if rec['Venue_Category1'] == 'bar':\n        iconi = 'glass-cheers'\n        prefixi = 'fa'\n        isIcon = True\n    if rec['Venue_Category1'] == 'lodging':\n        iconi = 'bed'\n        prefixi = 'fa'\n        isIcon = True\n    if rec['Venue_Category1'] == 'pharmacy':\n        iconi = 'prescription-bottle-alt'\n        prefixi = 'fa'\n        isIcon = True\n    if rec['Venue_Category1'] == 'lawyer':\n        iconi = 'gavel'\n        prefixi = 'fa'\n        isIcon = True\n    if rec['Venue_Category1'] == 'bakery':\n        iconi = 'bread-slice'\n        prefixi = 'fa'\n        isIcon = True\n    if rec['Venue_Categor

In [263]:
map_buzt

In [224]:
nearby_venues_buzt['Venue_Category1'].unique()

array(['locality', 'lodging', 'bank', 'supermarket', 'point_of_interest',
       'car_repair', 'health', 'bar', 'lawyer', 'pharmacy',
       'beauty_salon', 'electronics_store', 'general_contractor',
       'school', 'church', 'finance', 'painter', 'accounting', 'bakery',
       'hair_care', 'dentist', 'real_estate_agency'], dtype=object)